# Downstream Analysis

This notebook should be used for downstream analysis of your OPS screen.
Cells marked with <font color='red'>SET PARAMETERS</font> contain crucial variables that need to be set according to your specific experimental setup and data organization.
Please review and modify these variables as needed before proceeding with the analysis.

## <font color='red'>SET PARAMETERS</font>

### Fixed parameters for cluster module

- `CONFIG_FILE_PATH`: Path to a Brieflow config file used during processing. Absolute or relative to where workflows are run from.

In [1]:
CONFIG_FILE_PATH = "config/config.yml"

In [ ]:
from pathlib import Path
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
import yaml
import pandas as pd
import matplotlib as plt

from lib.cluster.cluster_analysis import (
    differential_analysis,
    waterfall_plot,
    two_feature_plot,
    cluster_heatmap,
)
from lib.cluster.phate_leiden_clustering import plot_phate_leiden_clusters
from lib.shared.metrics import get_all_stats

In [3]:
# load config file and determine root path
with open(CONFIG_FILE_PATH, "r") as config_file:
    config = yaml.safe_load(config_file)
    ROOT_FP = Path(config["all"]["root_fp"])

## Pipeline Statistics Report

This function analyzes the entire data processing pipeline from raw microscopy images to clustered perturbation profiles. It will:

1. **Preprocessing**: Count input files and generated image tiles
2. **SBS**: Analyze cell segmentation and barcode mapping success rates
3. **Phenotype**: Count cells and morphological features extracted
4. **Merge**: Calculate how many cells were successfully matched between SBS and phenotype data
5. **Aggregation**: Measure perturbation coverage and batch effect correction (per cell class/channel)
6. **Clustering**: Evaluate pathway enrichment and optimal clustering parameters

**Expected runtime: 5-15 minutes** depending on dataset size, as this function loads and analyzes large parquet files to calculate batch statistics and enrichment metrics.

The report will display progress updates as each stage is analyzed.

In [ ]:
statistics = get_all_stats(config)

In [4]:
# Load cell classes and channel combos
cluster_combo_fp = config["cluster"]["cluster_combo_fp"]
cluster_combos = pd.read_csv(cluster_combo_fp, sep="\t")

CHANNEL_COMBOS = list(cluster_combos["channel_combo"].unique())
print(f"Channel Combos: {CHANNEL_COMBOS}")

CELL_CLASSES = list(cluster_combos["cell_class"].unique())
print(f"Cell classes: {CELL_CLASSES}")

LEIDEN_RESOLUTION = list(cluster_combos["leiden_resolution"].unique())
print(f"Leiden resolution: {LEIDEN_RESOLUTION}")

Channel Combos: ['DAPI_COXIV_CENPA_WGA', 'DAPI_CENPA', 'DAPI_WGA']
Cell classes: ['Interphase', 'Mitotic', 'all']
Leiden resolution: [np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(15)]


## <font color='red'>SET PARAMETERS</font>

### Cluster Selection for Analysis

Set these parameters to select the specific cluster data to analyze:
- `CHANNEL_COMBO`: Select from available channel combinations,
- `CELL_CLASS`: Select from available cell classes,
- `LEIDEN_RESOLUTION`: Select from available Leiden resolutions,

These parameters determine which folder of cluster data will be analyzed.

In [5]:
CHANNEL_COMBO = "DAPI_CENPA"
CELL_CLASS = "Interphase"
LEIDEN_RESOLUTION = 15

In [ ]:
aggregate_file = (
    ROOT_FP
    / "aggregate"
    / "tsvs"
    / f"CeCl-{CELL_CLASS}_ChCo-{CHANNEL_COMBO}__feature_table.tsv"
)
print(f"Aggregate file: {aggregate_file}")

if not aggregate_file.exists():
    print(f"Aggregate file does not exist: {aggregate_file}")
else:
    print(f"Aggregate file found")

cluster_path = ROOT_FP / "cluster" / CHANNEL_COMBO / CELL_CLASS / str(LEIDEN_RESOLUTION)
print(f"Cluster path: {cluster_path}")

if not cluster_path.exists():
    print(f"Cluster directory does not exist: {cluster_path}")
else:
    print(f"Cluster directory found")

Aggregate file: brieflow_output/aggregate/tsvs/CeCl-Interphase_ChCo-DAPI_CENPA__feature_table.tsv
Aggregate file found
Cluster path: brieflow_output/cluster/DAPI_CENPA/Interphase/15
Cluster directory found


## Feature Plot Analysis

This section generates visualizations to explore the phenotypic effects of gene perturbations in your screen. The plots will help you:

1. **Differential Feature Analysis**: Identify genes with significant phenotypic changes vs. controls
2. **Waterfall Plots**: Rank genes by their effect on specific features of interest
3. **Two-Feature Plots**: Discover relationships between different phenotypic measurements
4. **Heatmaps**: Visualize patterns across multiple features and gene sets simultaneously

The interactive analysis allow you to customize each visualization for your specific biological questions.

In [7]:
cluster_file = cluster_path / "phate_leiden_clustering.tsv"
cluster_df = pd.read_csv(cluster_file, sep="\t")
display(cluster_df)

,gene_symbol_0,cell_count,PHATE_0,PHATE_1,cluster,uniprot_entry,uniprot_function,uniprot_link,mean_potential_to_nontargeting,normalized_potential_to_nontargeting
0,ARL17A,1798,0.001913,-0.008636,0,Q8IVW1,GTP-binding protein that functions as an allos...,https://www.uniprot.org/uniprotkb/Q8IVW1/entry,1.524561,0.092283
1,NMT1,1535,0.005146,-0.008961,0,P30419,Adds a myristoyl group to the N-terminal glyci...,https://www.uniprot.org/uniprotkb/P30419/entry,1.618714,0.099207
2,RAD51D,2159,0.003042,-0.006375,0,O75771,Involved in the homologous recombination repai...,https://www.uniprot.org/uniprotkb/O75771/entry,1.328814,0.077889
3,PDK2,2170,0.000592,-0.009447,0,Q15119,Kinase that plays a key role in the regulation...,https://www.uniprot.org/uniprotkb/Q15119/entry,1.620593,0.099345
4,ADAM11,3157,0.002060,-0.008379,0,O75078,Probable ligand for integrin in the brain. Thi...,https://www.uniprot.org/uniprotkb/O75078/entry,1.529272,0.092630
...,...,...,...,...,...,...,...,...,...,...
5294,MPHOSPH6,1937,-0.006584,-0.001020,413,Q99547,RNA-binding protein that associates with the R...,https://www.uniprot.org/uniprotkb/Q99547/entry,1.035970,0.056356
5295,RFK,3317,-0.000036,0.004026,413,Q969G6,Catalyzes the phosphorylation of riboflavin (v...,https://www.uniprot.org/uniprotkb/Q969G6/entry,0.315340,0.003365
5296,CWF19L1,1771,-0.016267,0.000340,413,Q69YN2,NaN,https://www.uniprot.org/uniprotkb/Q69YN2/entry,1.938802,0.122744
5297,BRD1,2056,0.000171,0.005380,413,O95696,Scaffold subunit of various histone acetyltran...,https://www.uniprot.org/uniprotkb/O95696/entry,0.296676,0.001993


In [8]:
aggregate_df = pd.read_csv(aggregate_file, sep="\t")
display(aggregate_df)

,gene_symbol_0,nucleus_DAPI_int,nucleus_CENPA_int,nucleus_DAPI_mean,nucleus_CENPA_mean,nucleus_DAPI_mean_edge,nucleus_CENPA_mean_edge,nucleus_DAPI_std_edge,nucleus_CENPA_std_edge,nucleus_DAPI_mass_displacement,...,cell_area,cell_convex_area,cell_form_factor,cell_solidity,cell_eccentricity,cell_CENPA_foci_area,nucleus_manders_DAPI_CENPA,nucleus_manders_CENPA_DAPI,cell_manders_DAPI_CENPA,cell_manders_CENPA_DAPI
0,MED16,-0.431912,-0.054273,-0.295063,-0.223577,-0.267605,-0.219356,-0.284762,-0.193909,-0.128088,...,0.017105,0.019299,0.223874,0.223454,0.137092,-0.051670,-0.007315,-0.077321,-0.064443,0.076250
1,PSEN2,-0.437285,-0.058696,-0.250357,-0.134306,-0.190773,-0.130073,-0.207578,-0.173246,-0.123685,...,-0.046368,-0.047886,0.233865,0.186801,0.090084,-0.092773,-0.022021,-0.073659,0.015532,-0.019700
2,ZNRD2,-0.454941,-0.183587,-0.209065,-0.127330,-0.196158,-0.115154,-0.183905,-0.196196,-0.125362,...,-0.148400,-0.153234,0.178303,0.167947,0.179500,-0.051670,0.070928,-0.000687,0.072241,-0.025813
3,TUBA4A,-0.433277,-0.065066,-0.194619,-0.118311,-0.163329,-0.121516,-0.161941,-0.198220,-0.133639,...,-0.067439,-0.067791,0.232195,0.197662,0.150687,-0.092773,0.054688,0.017595,0.035148,-0.022518
4,SERINC4,-0.451554,-0.090615,-0.260085,-0.145412,-0.218946,-0.131034,-0.193756,-0.162884,-0.101623,...,-0.089195,-0.096004,0.212671,0.189382,0.162972,-0.051670,0.090022,-0.063655,0.040556,0.071686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5294,nontargeting_GTGACGCCGGT_GTGACGCCGGTCCATGCGAT,-0.459858,-0.041368,-0.353813,-0.262070,-0.236404,-0.175133,-0.314674,-0.258213,0.139595,...,-0.111549,-0.101791,0.189923,0.184277,0.109802,-0.025766,-0.214820,-0.133163,-0.201888,0.136544
5295,nontargeting_GCGGCGTCATC_GCGGCGTCATCCACGCGTGC,-0.407634,0.299336,0.353688,0.285284,0.263272,0.281667,0.124270,-0.026239,-0.126121,...,0.123493,0.107860,0.322060,0.311753,-0.116043,-0.026279,0.005896,-0.299790,0.394311,-0.219372
5296,nontargeting_GATCGTGGTTA_GATCGTGGTTAACTCCGACC,-0.506576,0.194775,-0.471700,-0.113419,-0.381416,-0.093443,-0.285469,-0.151733,-0.125160,...,0.025836,0.026256,0.270637,0.183950,0.150555,-0.026279,0.161548,-0.149520,-0.191902,0.088476
5297,nontargeting_CGGCTTTGTTG_CGGCTTTGTTGCCCGTAAGC,-0.455211,-0.203548,-0.279813,-0.159765,-0.163336,-0.162170,-0.347177,-0.162180,-0.079574,...,-0.186844,-0.172850,0.286480,0.264899,-0.012559,-0.026499,-0.190248,-0.119814,-0.220742,0.148236


## <font color='red'>SET PARAMETERS</font>

### Cluster Selection for Visualization

Set these parameters to select the specific cluster to analyze:
- `CLUSTER_ID`: The cluster of interest to generate plots.

In [ ]:
CLUSTER_ID = None

In [ ]:
print(f"\n{'=' * 50}")
print(f"Analyzing Cluster {CLUSTER_ID}")
cluster_genes = cluster_df[cluster_df["cluster"] == CLUSTER_ID][
    config["aggregate"]["perturbation_name_col"]
]
print(f"Genes in Cluster {CLUSTER_ID}:", ", ".join(cluster_genes.unique()))
print(f"{'=' * 50}")

# Run differential analysis using robust z-score
diff_results = differential_analysis(
    feature_df=aggregate_df,
    cluster_df=cluster_df,
    cluster_id=CLUSTER_ID,
    control_type="nontargeting",
    control_label=config["aggregate"]["control_key"],
    use_nonparametric=True,
    normalize_method="robust_zscore",
)

# Display results in a more user-friendly format
print("\nTop Upregulated Features:")
up_df = diff_results["top_up"][
    ["feature", "robust_zscore", "p_value", "median_test", "median_control"]
]
up_df = up_df.rename(
    columns={
        "robust_zscore": "Z-score",
        "p_value": "p-value",
        "median_test": "Median (test)",
        "median_control": "Median (control)",
    }
)
display(
    up_df.style.format(
        {
            "Z-score": "{:.2f}",
            "p-value": "{:.2e}",
            "Median (test)": "{:.3f}",
            "Median (control)": "{:.3f}",
        }
    )
)

print("\nTop Downregulated Features:")
down_df = diff_results["top_down"][
    ["feature", "robust_zscore", "p_value", "median_test", "median_control"]
]
down_df = down_df.rename(
    columns={
        "robust_zscore": "Z-score",
        "p_value": "p-value",
        "median_test": "Median (test)",
        "median_control": "Median (control)",
    }
)
display(
    down_df.style.format(
        {
            "Z-score": "{:.2f}",
            "p-value": "{:.2e}",
            "Median (test)": "{:.3f}",
            "Median (control)": "{:.3f}",
        }
    )
)

## <font color='red'>SET PARAMETERS</font>

### Feature Selection for Visualization

Set these parameters to select the specific cluster to analyze:
- `FEATURES_TO_ANALYZE`: The features found in the differential analysis to generate plots for.
- `GENES_TO_LABEL`: The genes within the cluster to label on the plots.

In [ ]:
FEATURES_TO_ANALYZE = None
GENES_TO_LABEL = None

In [ ]:
# Create waterfall plots for selected features
print("\n--- Waterfall Plots for Selected Features ---")
for feature in FEATURES_TO_ANALYZE:
    print(f"\nPlotting: {feature}")
    waterfall_plot(
        feature_df=aggregate_df,
        feature=feature,
        cluster_df=cluster_df,
        cluster_id=CLUSTER_ID,
        nontargeting_pattern=config["aggregate"]["control_key"],
        title=f"Cluster {CLUSTER_ID}: {feature}",
        label_genes=GENES_TO_LABEL,
    )

# Create two-feature plots for combinations
print("\n--- Two-Feature Plots ---")
# Generate all unique pairs of features
feature_pairs = [
    (FEATURES_TO_ANALYZE[i], FEATURES_TO_ANALYZE[j])
    for i in range(len(FEATURES_TO_ANALYZE))
    for j in range(i + 1, len(FEATURES_TO_ANALYZE))
]

for feature1, feature2 in feature_pairs:
    print(f"\nPlotting: {feature1} vs {feature2}")
    two_feature_plot(
        feature_df=aggregate_df,
        x=feature1,
        y=feature2,
        cluster_df=cluster_df,
        cluster_id=CLUSTER_ID,
        nontargeting_pattern=config["aggregate"]["control_key"],
        title=f"Cluster {CLUSTER_ID}: {feature1} vs {feature2}",
        label_genes=GENES_TO_LABEL,
    )

# Create heatmap with all differential features
print("\n--- Heatmap of Differential Features ---")
# Get top features from differential analysis
top_up = (
    diff_results["top_up"]["feature"].tolist()
    if not diff_results["top_up"].empty
    else []
)
top_down = (
    diff_results["top_down"]["feature"].tolist()
    if not diff_results["top_down"].empty
    else []
)
diff_features = top_up + top_down

fig, ax, heatmap_data = cluster_heatmap(
    feature_df=aggregate_df,
    cluster_df=cluster_df,
    cluster_ids=[CLUSTER_ID],
    features=diff_features,
    perturbation_name_col=config["aggregate"]["perturbation_name_col"],
    z_score="global",
    title=f"Cluster {CLUSTER_ID}: Top Differential Features",
)

# PHATE plot for the cluster
print("\n--- PHATE Plot for Cluster ---")
phate_fig = plot_phate_leiden_clusters(
    phate_leiden_clustering=cluster_df,
    perturbation_name_col=config["aggregate"]["perturbation_name_col"],
    control_key=config["aggregate"]["control_key"],
    clusters_of_interest=[CLUSTER_ID],
)

## Mozzarellm: LLM-based Gene Cluster Analysis

### Overview
[Mozzarellm](https://github.com/cheeseman-lab/mozzarellm) is a Python package that leverages Large Language Models (LLMs) to analyze gene clusters for pathway identification and novel gene discovery. This notebook guides you through the process of:

1. **Loading and reshaping gene cluster data** from your OPS screen
2. **Analyzing gene clusters with LLMs** to identify biological pathways
3. **Categorizing genes** as established pathway members, uncharacterized, or having novel potential roles
4. **Prioritizing candidates** for experimental validation

### Prerequisites

You need to install the mozzarellm package in your Brieflow environment:

```bash
pip install git+https://github.com/cheeseman-lab/mozzarellm.git
```

Mozzarellm requires API keys to access LLM services. You need at least one of these keys:

- **OpenAI API Key**: Required for GPT models (gpt-4o, gpt-4.5, etc.)
- **Anthropic API Key**: Required for Claude models (claude-3-7-sonnet, etc.)
- **Google API Key**: Required for Gemini models (gemini-2.0-pro, etc.)

These keys provide access to paid API services, and usage will incur costs based on the number of tokens processed. The cost per analysis varies by model but typically ranges from $0.01-$0.10 per cluster, depending on cluster size and model choice. For this reason, we only run these analyses on a specific chosen Leiden resolution, rather than across all of the generated possible resolutions.

In [9]:
import os
from mozzarellm import analyze_gene_clusters, reshape_to_clusters

In [ ]:
# Set API keys
os.environ["OPENAI_API_KEY"] = "your_openai_key_here"
os.environ["ANTHROPIC_API_KEY"] = "your_anthropic_key_here"
os.environ["GOOGLE_API_KEY"] = "your_google_key_here"

In [12]:
!pwd

/lab/ops_analysis/cheeseman/denali-analysis/analysis


In [ ]:
RESULTS_DIR = Path(
    "/lab/ops_analysis/cheeseman/denali-temp/mozzarellm_analysis/"
)  # cluster_path / "mozzarellm_analysis"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
print(f"Results will be saved to: {RESULTS_DIR}")

Results will be saved to: /lab/ops_analysis/cheeseman/denali-temp/mozzarellm_analysis


In [15]:
cluster_file = cluster_path / "phate_leiden_clustering.tsv"
cluster_df = pd.read_csv(cluster_file, sep="\t")
display(cluster_df)

,gene_symbol_0,cell_count,PHATE_0,PHATE_1,cluster,uniprot_entry,uniprot_function,uniprot_link,mean_potential_to_nontargeting,normalized_potential_to_nontargeting
0,ARL17A,1798,0.001913,-0.008636,0,Q8IVW1,GTP-binding protein that functions as an allos...,https://www.uniprot.org/uniprotkb/Q8IVW1/entry,1.524561,0.092283
1,NMT1,1535,0.005146,-0.008961,0,P30419,Adds a myristoyl group to the N-terminal glyci...,https://www.uniprot.org/uniprotkb/P30419/entry,1.618714,0.099207
2,RAD51D,2159,0.003042,-0.006375,0,O75771,Involved in the homologous recombination repai...,https://www.uniprot.org/uniprotkb/O75771/entry,1.328814,0.077889
3,PDK2,2170,0.000592,-0.009447,0,Q15119,Kinase that plays a key role in the regulation...,https://www.uniprot.org/uniprotkb/Q15119/entry,1.620593,0.099345
4,ADAM11,3157,0.002060,-0.008379,0,O75078,Probable ligand for integrin in the brain. Thi...,https://www.uniprot.org/uniprotkb/O75078/entry,1.529272,0.092630
...,...,...,...,...,...,...,...,...,...,...
5294,MPHOSPH6,1937,-0.006584,-0.001020,413,Q99547,RNA-binding protein that associates with the R...,https://www.uniprot.org/uniprotkb/Q99547/entry,1.035970,0.056356
5295,RFK,3317,-0.000036,0.004026,413,Q969G6,Catalyzes the phosphorylation of riboflavin (v...,https://www.uniprot.org/uniprotkb/Q969G6/entry,0.315340,0.003365
5296,CWF19L1,1771,-0.016267,0.000340,413,Q69YN2,NaN,https://www.uniprot.org/uniprotkb/Q69YN2/entry,1.938802,0.122744
5297,BRD1,2056,0.000171,0.005380,413,O95696,Scaffold subunit of various histone acetyltran...,https://www.uniprot.org/uniprotkb/O95696/entry,0.296676,0.001993


## <font color='red'>SET PARAMETERS</font>: 

### Data Reshaping

Configure gene clustering parameters:
- `GENE_COL`: Column containing gene identifiers
- `CLUSTER_COL`: Column containing cluster assignments
- `UNIPROT_COL`: Column with UniProt annotations

These parameters control how gene-level data is converted to cluster-level data. These are set to default values that will be established in upstream notebooks/snakemakes, and usually don't need to be manipulated.

In [16]:
GENE_COL = config["aggregate"]["perturbation_name_col"]
CLUSTER_COL = "cluster"
UNIPROT_COL = "uniprot_function"

In [ ]:
llm_cluster_df, llm_uniprot_df = reshape_to_clusters(
    input_df=cluster_df,
    gene_col=GENE_COL,
    cluster_col=CLUSTER_COL,
    uniprot_col=UNIPROT_COL,
    verbose=True,
)
display(llm_cluster_df)
display(llm_uniprot_df)

Using provided DataFrame with 5299 rows
Found 5299 genes across 414 clusters
Extracting gene features from uniprot_function column


,cluster_id,genes
0,0,ARL17A;NMT1;RAD51D;PDK2;ADAM11;ACACA;TBC1D3H;T...
1,1,UBL5;CDC5L;PRPF40A;SMN1;RBM42;SNRPE;SNRPD1;SNR...
2,2,BCAR1;SEC62;CBWD6;RPN2;SFPQ;PTPN1;SEC61A1;TLN1...
3,3,WDHD1;SKP2;POLE;LRR1;PCNA;CDC7;CDC6;GEMIN7;PRI...
4,4,RBM19;BUD23;DDX47;KIFBP;RRP36;EXOSC6;ESF1;NOL6...
...,...,...
409,409,SGCA;DLD;SKAP1;SPAG9;HEATR6
410,410,MYL12A;TM7SF2;SFTPB;RCE1;PTDSS1
411,411,LMNA;HNRNPA3;FAM83H;LIN7C;PRMT5
412,412,MRPS18B;GLRX5;LRCH4;MAGOHB;GNG3


,gene_symbol_0,uniprot_function
0,ARL17A,GTP-binding protein that functions as an allos...
1,NMT1,Adds a myristoyl group to the N-terminal glyci...
2,RAD51D,Involved in the homologous recombination repai...
3,PDK2,Kinase that plays a key role in the regulation...
4,ADAM11,Probable ligand for integrin in the brain. Thi...
...,...,...
5294,MPHOSPH6,RNA-binding protein that associates with the R...
5295,RFK,Catalyzes the phosphorylation of riboflavin (v...
5296,CWF19L1,
5297,BRD1,Scaffold subunit of various histone acetyltran...


## <font color='red'>SET PARAMETERS</font>: 
### LLM Analysis Configuration

- `MODEL_NAME`: LLM to use for analysis. Usable models include:
  - OpenAI: `o4-mini`, `o3-mini`, `gpt-4.1`, `gpt-4o`
  - Anthropic: `claude-3-7-sonnet-20250219`, `claude-3-5-haiku-20241022`
  - Google: `gemini-2.5-pro-preview-03-25`, `gemini-2.5-flash-preview-04-17`
- `CONFIG_DICT`: Configuration file for the LLM model
- `SCREEN_CONTEXT`: Context for the analysis and how to evaluate _clusters_
- `CLUSTER_ANALYSIS_PROMPT`: Context for the analysis and how to evaluate _genes within clusters_

Mozzarellm includes optimized [configurations](https://github.com/cheeseman-lab/mozzarellm/blob/main/mozzarellm/configs.py) and [prompts](https://github.com/cheeseman-lab/mozzarellm/blob/main/mozzarellm/prompts.py) you can import as shown below.

Custom text files can also be used by setting `screen_context_path` and `cluster_analysis_prompt_path` parameters.

In [18]:
from mozzarellm.prompts import ROBUST_SCREEN_CONTEXT, ROBUST_CLUSTER_PROMPT
from mozzarellm.configs import DEFAULT_ANTHROPIC_CONFIG

In [19]:
# Set up model configs
MODEL_NAME = "claude-3-7-sonnet-20250219"
CONFIG_DICT = DEFAULT_ANTHROPIC_CONFIG
SCREEN_CONTEXT = ROBUST_SCREEN_CONTEXT
CLUSTER_ANALYSIS_PROMPT = ROBUST_CLUSTER_PROMPT

In [20]:
# Run LLM analysis
anthropic_results = analyze_gene_clusters(
    # Input data options
    input_df=llm_cluster_df,
    # Model and configuration
    model_name=MODEL_NAME,
    config_dict=CONFIG_DICT,
    # Analysis context and prompts
    screen_context=SCREEN_CONTEXT,
    cluster_analysis_prompt=CLUSTER_ANALYSIS_PROMPT,
    # Gene annotations
    gene_annotations_df=llm_uniprot_df,
    # Processing options
    batch_size=1,
    # Output options
    output_file=f"{RESULTS_DIR}/{MODEL_NAME}",
    save_outputs=True,
    outputs_to_generate=["json", "clusters", "flagged_genes"],
)

Loaded data with 414 rows and columns: ['cluster_id', 'genes']
Created annotations dictionary with 5299 entries from DataFrame


Processing clusters:   0%|          | 0/414 [00:00<?, ?it/s]

Using provided template string
Appending output format instructions to template
Added 65 gene feature descriptions to prompt


Processing clusters:   0%|          | 1/414 [00:17<2:03:29, 17.94s/it]

Using provided template string
Appending output format instructions to template
Added 62 gene feature descriptions to prompt


...

INFO:cluster_analysis_20250727_221630.log:Anthropic API call successful: model=claude-3-7-sonnet-20250219
INFO:cluster_analysis_20250727_221630.log:Success for cluster 411
Processing clusters: 100%|█████████▉| 412/414 [1:45:25<00:21, 10.70s/it]INFO:cluster_analysis_20250727_221630.log:Using Anthropic Claude API


Using provided template string
Appending output format instructions to template
Added 5 gene feature descriptions to prompt


INFO:cluster_analysis_20250727_221630.log:Anthropic API call successful: model=claude-3-7-sonnet-20250219
INFO:cluster_analysis_20250727_221630.log:Success for cluster 412
Processing clusters: 100%|█████████▉| 413/414 [1:45:35<00:10, 10.37s/it]INFO:cluster_analysis_20250727_221630.log:Using Anthropic Claude API


Using provided template string
Appending output format instructions to template
Added 5 gene feature descriptions to prompt


INFO:cluster_analysis_20250727_221630.log:Anthropic API call successful: model=claude-3-7-sonnet-20250219
INFO:cluster_analysis_20250727_221630.log:Success for cluster 413
Processing clusters: 100%|██████████| 414/414 [1:45:48<00:00, 15.33s/it]
INFO:cluster_analysis_20250727_221630.log:Completed analysis for 414 clusters with data saved to disk


## Cluster overlap analysis

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter


def analyze_cluster_novelty(
    clustering_df,
    aconcagua_df,
    enrichment_df,
    mozzarellm_df,
    gene_col="gene_symbol_0",
    new_cluster_col="cluster",
    old_cluster_col="cluster",
    jaccard_threshold=0.3,
    target_keywords=None,
):
    """

    # Default keywords for kinetochore/CENPT screen
    if target_keywords is None:
        target_keywords = ['kinetochore', 'centromere', 'chromosome', 'mitosis', 'spindle', 'CENP',
                          'segregation', 'cohesion', 'condensin', 'aurora', 'polo', 'checkpoint']

    # Convert keywords to lowercase for case-insensitive matching
    target_keywords = [kw.lower() for kw in target_keywords]
    Analyze novelty of new clusters compared to ground truth clustering.

    Parameters:
    -----------
    clustering_df : pd.DataFrame
        New clustering results with columns: gene_symbol_0, cluster, etc.
    aconcagua_df : pd.DataFrame
        Ground truth clustering with columns: gene_symbol_0, cluster, etc.
    enrichment_df : pd.DataFrame
        KEGG/CORUM enrichment results with columns: cluster, KEGG_enriched_groups, CORUM_enriched_groups, etc.
    mozzarellm_df : pd.DataFrame
        LLM analysis results with columns: cluster_id, cluster_biological_process, etc.
    gene_col : str
        Column name for genes (default: 'gene_symbol_0')
    new_cluster_col : str
        Column name for new cluster assignments (default: 'cluster')
    old_cluster_col : str
        Column name for old cluster assignments (default: 'cluster')
    jaccard_threshold : float
        Threshold for considering a cluster "novel" (default: 0.3)
    target_keywords : list
        Keywords to search for in biological_process and follow_up_suggestion
        (default: ['kinetochore', 'centromere', 'chromosome', 'mitosis', 'spindle', 'CENP'])

    Returns:
    --------
    pd.DataFrame
        Analysis results sorted by novelty metrics
    """

    # Merge new clustering with ground truth
    merged_data = clustering_df.merge(
        aconcagua_df, on=gene_col, how="inner", suffixes=("_new", "_old")
    )

    if merged_data.empty:
        print("Warning: No overlapping genes found between datasets")
        return pd.DataFrame()

    print(f"Found {len(merged_data)} overlapping genes")

    # Get unique clusters
    new_clusters = merged_data[f"{new_cluster_col}_new"].unique()
    old_clusters = merged_data[f"{old_cluster_col}_old"].unique()

    results = []

    for new_cluster in new_clusters:
        # Get genes in this new cluster
        new_cluster_genes = set(
            merged_data[merged_data[f"{new_cluster_col}_new"] == new_cluster][gene_col]
        )

        # Calculate Jaccard index with all old clusters
        jaccard_scores = {}
        for old_cluster in old_clusters:
            old_cluster_genes = set(
                merged_data[merged_data[f"{old_cluster_col}_old"] == old_cluster][
                    gene_col
                ]
            )

            intersection = len(new_cluster_genes.intersection(old_cluster_genes))
            union = len(new_cluster_genes.union(old_cluster_genes))

            if union > 0:
                jaccard_scores[old_cluster] = intersection / union
            else:
                jaccard_scores[old_cluster] = 0

        # Find best matching old cluster
        max_jaccard = max(jaccard_scores.values()) if jaccard_scores else 0
        best_old_cluster = (
            max(jaccard_scores, key=jaccard_scores.get) if jaccard_scores else None
        )

        # Get genes from the closest correlated cluster
        closest_cluster_genes = set()
        if best_old_cluster is not None:
            closest_cluster_genes = set(
                merged_data[merged_data[f"{old_cluster_col}_old"] == best_old_cluster][
                    gene_col
                ]
            )

        # Calculate average normalized_potential_to_nontargeting for this cluster
        cluster_potential_scores = clustering_df[
            clustering_df[new_cluster_col] == new_cluster
        ]["normalized_potential_to_nontargeting"]
        avg_potential_score = (
            cluster_potential_scores.mean() if len(cluster_potential_scores) > 0 else 0
        )

        # Calculate entropy of old cluster composition
        old_cluster_composition = merged_data[
            merged_data[f"{new_cluster_col}_new"] == new_cluster
        ][f"{old_cluster_col}_old"].value_counts()
        if len(old_cluster_composition) > 1:
            proportions = old_cluster_composition / old_cluster_composition.sum()
            entropy = -sum(p * np.log2(p) for p in proportions if p > 0)
        else:
            entropy = 0

        # Get enrichment data for this cluster
        enrichment_data = enrichment_df[enrichment_df["cluster"] == new_cluster]

        # Extract enrichment information
        kegg_terms = ""
        corum_terms = ""
        string_precision = 0
        cluster_size_enrichment = 0

        if not enrichment_data.empty:
            row = enrichment_data.iloc[0]  # Take first row if multiple
            kegg_terms = (
                str(row.get("KEGG_enriched_groups", ""))
                .replace("nan", "")
                .replace("NaN", "")
            )
            corum_terms = (
                str(row.get("CORUM_enriched_groups", ""))
                .replace("nan", "")
                .replace("NaN", "")
            )
            string_precision = (
                row.get("STRING_precision", 0)
                if pd.notna(row.get("STRING_precision", 0))
                else 0
            )
            cluster_size_enrichment = (
                row.get("cluster_size", 0)
                if pd.notna(row.get("cluster_size", 0))
                else 0
            )

        # Get LLM analysis for this cluster
        biological_process = ""
        pathway_confidence = ""
        importance_score = 0
        follow_up = ""
        established_genes = ""
        uncharacterized_genes = ""
        novel_role_genes = ""
        percent_established = 0

        llm_data = mozzarellm_df[mozzarellm_df["cluster_id"] == new_cluster]
        if not llm_data.empty:
            row = llm_data.iloc[0]  # Take first row if multiple
            biological_process = (
                str(row.get("cluster_biological_process", ""))
                .replace("nan", "")
                .replace("NaN", "")
            )
            pathway_confidence = (
                str(row.get("pathway_confidence_level", ""))
                .replace("nan", "")
                .replace("NaN", "")
            )
            importance_score = (
                row.get("cluster_importance_score", 0)
                if pd.notna(row.get("cluster_importance_score", 0))
                else 0
            )
            follow_up = (
                str(row.get("follow_up_suggestion", ""))
                .replace("nan", "")
                .replace("NaN", "")
            )
            established_genes = (
                str(row.get("established_genes", ""))
                .replace("nan", "")
                .replace("NaN", "")
            )
            uncharacterized_genes = (
                str(row.get("uncharacterized_genes", ""))
                .replace("nan", "")
                .replace("NaN", "")
            )
            novel_role_genes = (
                str(row.get("novel_role_genes", ""))
                .replace("nan", "")
                .replace("NaN", "")
            )

            # Calculate percent established genes
            established_count = (
                row.get("established_gene_count", 0)
                if pd.notna(row.get("established_gene_count", 0))
                else 0
            )
            total_count = (
                row.get("total_gene_count", 0)
                if pd.notna(row.get("total_gene_count", 0))
                else 0
            )
            percent_established = (
                (established_count / total_count * 100) if total_count > 0 else 0
            )

        # Search for target keywords
        keywords_found = []
        search_text = f"{biological_process} {follow_up}".lower()

        for keyword in target_keywords:
            if keyword in search_text:
                keywords_found.append(keyword)

        keywords_found_str = "; ".join(keywords_found) if keywords_found else ""

        # Compile results
        result = {
            "new_cluster": new_cluster,
            "cluster_size": len(new_cluster_genes),
            "avg_potential_score": avg_potential_score,
            "max_jaccard": max_jaccard,
            "best_old_cluster": best_old_cluster,
            "entropy": entropy,
            "is_novel": max_jaccard < jaccard_threshold,
            "string_precision": string_precision,
            "kegg_terms": kegg_terms if kegg_terms != "nan" else "",
            "corum_terms": corum_terms if corum_terms != "nan" else "",
            "biological_process": biological_process
            if biological_process != "nan"
            else "",
            "pathway_confidence": pathway_confidence
            if pathway_confidence != "nan"
            else "",
            "importance_score": importance_score,
            "percent_established": percent_established,
            "follow_up_suggestion": follow_up if follow_up != "nan" else "",
            "keywords_found": keywords_found_str,
            "established_genes": established_genes
            if established_genes != "nan"
            else "",
            "uncharacterized_genes": uncharacterized_genes
            if uncharacterized_genes != "nan"
            else "",
            "novel_role_genes": novel_role_genes if novel_role_genes != "nan" else "",
            "genes": ",".join(sorted(new_cluster_genes)),
            "closest_cluster_genes": ",".join(sorted(closest_cluster_genes)),
        }

        results.append(result)

    # Convert to DataFrame and sort by novelty
    results_df = pd.DataFrame(results)

    # Sort by: novelty first, then by entropy (higher = more mixed origins), then by importance
    results_df = results_df.sort_values(
        ["is_novel", "max_jaccard", "entropy", "importance_score"],
        ascending=[False, True, False, False],
    )

    # Add rank for easy reference
    results_df.reset_index(drop=True, inplace=True)
    results_df["novelty_rank"] = range(1, len(results_df) + 1)

    print(f"\nAnalysis complete!")
    print(f"Total clusters analyzed: {len(results_df)}")
    print(
        f"Novel clusters (Jaccard < {jaccard_threshold}): {sum(results_df['is_novel'])}"
    )
    print(f"Clusters with target keywords: {sum(results_df['keywords_found'] != '')}")
    print(f"Average max Jaccard: {results_df['max_jaccard'].mean():.3f}")

    return results_df


# Helper function to save results
def save_cluster_results(results_df, output_prefix="cluster_novelty_analysis"):
    """
    Save cluster analysis results to CSV files.

    Parameters:
    -----------
    results_df : pd.DataFrame
        Results from analyze_cluster_novelty function
    output_prefix : str
        Prefix for output filenames (default: 'cluster_novelty_analysis')

    Returns:
    --------
    dict
        Dictionary with paths to saved files
    """
    saved_files = {}

    # Save all clusters
    all_file = f"{output_prefix}_all.csv"
    results_df.to_csv(all_file, index=False)
    saved_files["all_clusters"] = all_file
    print(f"Saved all clusters to: {all_file}")

    # Save keyword clusters only
    keyword_clusters = results_df[results_df["keywords_found"] != ""].copy()
    if not keyword_clusters.empty:
        keyword_file = f"{output_prefix}_keywords_only.csv"
        keyword_clusters.to_csv(keyword_file, index=False)
        saved_files["keyword_clusters"] = keyword_file
        print(f"Saved {len(keyword_clusters)} keyword clusters to: {keyword_file}")
    else:
        print("No clusters with target keywords found - keyword file not created")
        saved_files["keyword_clusters"] = None

    return saved_files

In [ ]:
clustering = (
    "brieflow_output/cluster/DAPI_CENPA/Interphase/15/phate_leiden_clustering.tsv"
)
enrichment = (
    "brieflow_output/cluster/DAPI_CENPA/Interphase/15/CB-Real__combined_table.tsv"
)
mozzarellm = "brieflow_output/cluster/DAPI_CENPA/Interphase/15/mozzarellm_analysis/claude-3-7-sonnet-20250219_clusters.csv"
aconcagua_clustering = "/lab/ops_analysis/cheeseman/aconcagua_clustering.csv"
clustering = pd.read_csv(clustering, sep="\t")
display(clustering)
enrichment = pd.read_csv(enrichment, sep="\t")
display(enrichment)
mozzarellm = pd.read_csv(mozzarellm)
display(mozzarellm)
aconcagua_clustering = pd.read_csv(aconcagua_clustering)
display(aconcagua_clustering)

,gene_symbol_0,cell_count,PHATE_0,PHATE_1,cluster,uniprot_entry,uniprot_function,uniprot_link,mean_potential_to_nontargeting,normalized_potential_to_nontargeting
0,ARL17A,1798,0.001913,-0.008636,0,Q8IVW1,GTP-binding protein that functions as an allos...,https://www.uniprot.org/uniprotkb/Q8IVW1/entry,1.524561,0.092283
1,NMT1,1535,0.005146,-0.008961,0,P30419,Adds a myristoyl group to the N-terminal glyci...,https://www.uniprot.org/uniprotkb/P30419/entry,1.618714,0.099207
2,RAD51D,2159,0.003042,-0.006375,0,O75771,Involved in the homologous recombination repai...,https://www.uniprot.org/uniprotkb/O75771/entry,1.328814,0.077889
3,PDK2,2170,0.000592,-0.009447,0,Q15119,Kinase that plays a key role in the regulation...,https://www.uniprot.org/uniprotkb/Q15119/entry,1.620593,0.099345
4,ADAM11,3157,0.002060,-0.008379,0,O75078,Probable ligand for integrin in the brain. Thi...,https://www.uniprot.org/uniprotkb/O75078/entry,1.529272,0.092630
...,...,...,...,...,...,...,...,...,...,...
5294,MPHOSPH6,1937,-0.006584,-0.001020,413,Q99547,RNA-binding protein that associates with the R...,https://www.uniprot.org/uniprotkb/Q99547/entry,1.035970,0.056356
5295,RFK,3317,-0.000036,0.004026,413,Q969G6,Catalyzes the phosphorylation of riboflavin (v...,https://www.uniprot.org/uniprotkb/Q969G6/entry,0.315340,0.003365
5296,CWF19L1,1771,-0.016267,0.000340,413,Q69YN2,NaN,https://www.uniprot.org/uniprotkb/Q69YN2/entry,1.938802,0.122744
5297,BRD1,2056,0.000171,0.005380,413,O95696,Scaffold subunit of various histone acetyltran...,https://www.uniprot.org/uniprotkb/O95696/entry,0.296676,0.001993


,cluster,genes,cluster_size,enriched_in,STRING_precision,STRING_true_positives,CORUM_enriched_groups,CORUM_enriched_ids,KEGG_enriched_groups,KEGG_enriched_ids
0,0,ACACA; ACLY; ADAM11; ALDOC; ARL17A; ATP6V0A1; ...,65,NaN,0.001923,4.0,NaN,NaN,NaN,NaN
1,1,AQP7; BCAS2; BRF2; CDC5L; CDK11A; DDX23; ECD; ...,62,"CORUM, STRING",0.308831,584.0,27.0,"(E.F.G) complex, 12S U11 snRNP, 17S U2 snRNP, ...",NaN,NaN
2,2,AKIRIN1; ALG2; ANKS1A; AXIN1; BCAR1; CARD10; C...,56,"CORUM, KEGG",0.030976,46.0,2.0,"PXN-ITGB5-PTK2 complex, Polycystin-1 multiprot...",7.0,KEGG_MEDICUS_PATHOGEN_HCMV_US27_TO_CXCR4_GNB_G...
3,3,BARD1; CCDC12; CDC45; CDC6; CDC7; CDT1; CHAF1A...,51,"CORUM, KEGG",0.061176,78.0,14.0,"DNA polymerase alpha-primase complex, DNA synt...",11.0,"KEGG_MEDICUS_REFERENCE_CORE_NER_REACTION, KEGG..."
4,4,AATF; ABT1; BMS1; BUD23; DBR1; DDX10; DDX18; D...,49,"CORUM, STRING",0.233844,275.0,2.0,"AATF-NGDN-NOL10 complex, Exosome",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
409,409,DLD; HEATR6; SGCA; SKAP1; SPAG9,5,NaN,0.000000,0.0,NaN,NaN,NaN,NaN
410,410,MYL12A; PTDSS1; RCE1; SFTPB; TM7SF2,5,NaN,0.000000,0.0,NaN,NaN,NaN,NaN
411,411,FAM83H; HNRNPA3; LIN7C; LMNA; PRMT5,5,NaN,0.000000,0.0,NaN,NaN,NaN,NaN
412,412,GLRX5; GNG3; LRCH4; MAGOHB; MRPS18B,5,NaN,0.000000,0.0,NaN,NaN,NaN,NaN


,cluster_id,cluster_biological_process,pathway_confidence_level,cluster_importance_score,follow_up_suggestion,established_genes,established_gene_count,uncharacterized_genes,uncharacterized_gene_count,novel_role_genes,...,total_gene_count,highest_unchar_importance,average_unchar_importance,high_unchar_genes,high_unchar_gene_count,highest_novel_role_importance,average_novel_role_importance,high_novel_role_genes,high_novel_role_gene_count,all_cluster_genes
0,0,Lipid metabolism and fatty acid biosynthesis,Medium,1.76,Cluster 0 shows a moderate enrichment for gene...,ACACA;ACLY;G6PC;PDK2;NMT1,5,C17orf58;SCRN2,2,BECN1;ALDOC;MED1;THRA,...,11,7,6.50,NaN,0,8,6.50,BECN1:8,1,ACACA;ACLY;G6PC;PDK2;NMT1;C17orf58;SCRN2;BECN1...
1,1,pre-mRNA splicing via spliceosome,High,2.97,This cluster shows a highly coherent signature...,CDC5L;PRPF8;SNRPD1;SNRPD2;SNRPD3;SNRPE;SNRPF;S...,36,RBM42,1,USPL1;TUT1;AQP7,...,40,6,6.00,NaN,0,9,7.33,AQP7:9,1,CDC5L;PRPF8;SNRPD1;SNRPD2;SNRPD3;SNRPE;SNRPF;S...
2,2,Integrin-mediated cell adhesion and focal adhe...,High,2.10,This cluster shows a strong signature of integ...,BCAR1;TLN1;ITGB1;ITGB5;PXN;ITGAV;FERMT2;PTK2;Z...,15,CBWD6;SUCO,2,SEC61A1;SEC61G;SEC61B;SLC39A7;TMED10;YAP1;TNS3,...,24,6,5.50,NaN,0,7,5.86,NaN,0,BCAR1;TLN1;ITGB1;ITGB5;PXN;ITGAV;FERMT2;PTK2;Z...
3,3,DNA replication and licensing,High,2.64,This cluster shows a highly coherent signature...,WDHD1;POLE;PCNA;CDC7;CDC6;PRIM1;DTL;CHAF1A;ORC...,24,PRR20D;CCDC12;ZNF705A;SYT15,4,MTBP;TRAIP;LRR1;DONSON;RTEL1;SLBP;NPAT;HINFP;F...,...,39,8,6.50,PRR20D:8,1,8,5.45,MTBP:8,1,WDHD1;POLE;PCNA;CDC7;CDC6;PRIM1;DTL;CHAF1A;ORC...
4,4,Ribosome biogenesis and RNA processing,High,3.24,Cluster 4 shows a strong and coherent signatur...,BMS1;BUD23;DIS3;ESF1;EXOSC1;EXOSC2;EXOSC3;EXOS...,30,KRI1;NOL10;NOC4L,3,ITGA4;KIFBP;DDX10;ABT1;DBR1;ZNHIT3;NOM1,...,40,7,6.33,NaN,0,9,6.57,ITGA4:9;KIFBP:8,2,BMS1;BUD23;DIS3;ESF1;EXOSC1;EXOSC2;EXOSC3;EXOS...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,409,Mitochondrial metabolism and energy production,Low,0.88,This cluster shows low confidence for a mitoch...,DLD,1,HEATR6,1,SGCA;SKAP1;SPAG9,...,5,8,8.00,HEATR6:8,1,7,6.00,NaN,0,DLD;HEATR6;SGCA;SKAP1;SPAG9
410,410,Membrane lipid biosynthesis and modification,Medium,1.40,This cluster reveals a membrane lipid biosynth...,TM7SF2;PTDSS1;RCE1,3,NaN,0,MYL12A;SFTPB,...,5,0,0.00,NaN,0,7,6.50,NaN,0,TM7SF2;PTDSS1;RCE1;MYL12A;SFTPB
411,411,Nuclear structure and chromatin organization,Medium,1.40,This cluster reveals a medium-confidence pathw...,LMNA;PRMT5,2,NaN,0,HNRNPA3;FAM83H;LIN7C,...,5,0,0.00,NaN,0,7,6.00,NaN,0,LMNA;PRMT5;HNRNPA3;FAM83H;LIN7C
412,412,Mitochondrial translation and ribosome assembly,Medium,1.40,This cluster shows a mitochondrial translation...,MRPS18B;GLRX5,2,NaN,0,MAGOHB;LRCH4;GNG3,...,5,0,0.00,NaN,0,7,6.00,NaN,0,MRPS18B;GLRX5;MAGOHB;LRCH4;GNG3


,gene_symbol_0,cell_count,PHATE_0,PHATE_1,cluster,uniprot_entry,uniprot_function,uniprot_link
0,SDHAF2,5431,0.002343,0.005769,0,Q9NX18,Plays an essential role in the assembly of suc...,https://www.uniprot.org/uniprotkb/Q9NX18/entry
1,FOLR3,8046,0.003512,0.007478,0,P41439,Binds to folate and reduced folic acid derivat...,https://www.uniprot.org/uniprotkb/P41439/entry
2,SCGB1D1,5083,0.003566,0.004751,0,O95968,"May bind androgens and other steroids, may als...",https://www.uniprot.org/uniprotkb/O95968/entry
3,PPP2R5B,5089,0.003158,0.007585,0,Q15173,As the regulatory component of the serine/thre...,https://www.uniprot.org/uniprotkb/Q15173/entry
4,RASGRP2,8026,0.002144,0.007154,0,Q7LDG7,Functions as a calcium- and DAG-regulated nucl...,https://www.uniprot.org/uniprotkb/Q7LDG7/entry
...,...,...,...,...,...,...,...,...
5294,TBRG4,5710,0.007220,0.005894,249,Q969Z0,Plays a role in processing of mitochondrial RN...,https://www.uniprot.org/uniprotkb/Q969Z0/entry
5295,MAGOHB,3399,0.009651,0.005044,250,Q96A72,Required for pre-mRNA splicing as component of...,https://www.uniprot.org/uniprotkb/Q96A72/entry
5296,RIOK1,3986,0.007534,0.003439,250,Q9BRS2,Involved in the final steps of cytoplasmic mat...,https://www.uniprot.org/uniprotkb/Q9BRS2/entry
5297,RBM18,5335,0.001253,0.006475,250,Q96H35,NaN,https://www.uniprot.org/uniprotkb/Q96H35/entry


In [ ]:
custom_keywords = [
    "kinetochore",
    "centromere",
    "CENP",
    "mitosis",
    "chromatin",
    "epigenetic",
]
results = analyze_cluster_novelty(
    clustering,
    aconcagua_clustering,
    enrichment,
    mozzarellm,
    target_keywords=custom_keywords,
)
saved_files = save_cluster_results(results, "cenpa_screen_results")

# View most novel clusters
print("Top 10 most novel clusters:")
print(
    results[results["is_novel"]].head(10)[
        [
            "new_cluster",
            "max_jaccard",
            "entropy",
            "biological_process",
            "keywords_found",
        ]
    ]
)

# View clusters with target keywords
print("Clusters with target keywords:")
print(
    results[results["keywords_found"] != ""][
        ["new_cluster", "keywords_found", "biological_process"]
    ]
)

Found 5031 overlapping genes

Analysis complete!
Total clusters analyzed: 414
Novel clusters (Jaccard < 0.3): 388
Clusters with target keywords: 97
Average max Jaccard: 0.104
Saved all clusters to: cenpa_screen_results_all.csv
Saved 97 keyword clusters to: cenpa_screen_results_keywords_only.csv
Top 10 most novel clusters:
   new_cluster  max_jaccard   entropy  \
0          405     0.026316  2.251629   
1          396     0.027778  2.584963   
2          397     0.039216  2.251629   
3          250     0.039604  2.521928   
4           57     0.040000  3.773557   
5           62     0.040000  3.773557   
6          409     0.040000  1.921928   
7           72     0.041667  3.521641   
8           90     0.043478  3.584963   
9           82     0.043478  3.546594   

                                  biological_process keywords_found  
0  Mitotic spindle regulation and cytoskeletal dy...        mitosis  
1     Autophagy regulation and vesicular trafficking                 
2       Centri